In [1]:
import mujoco
import numpy as np
import pinocchio as pin
from robot_descriptions.h1_description import URDF_PATH
from robot_descriptions.h1_mj_description import MJCF_PATH

from mujoco_sysid import parameters
from mujoco_sysid.utils import muj2pin

mjmodel = mujoco.MjModel.from_xml_path(MJCF_PATH)
mjdata = mujoco.MjData(mjmodel)

pinmodel = pin.buildModelFromUrdf(URDF_PATH, pin.JointModelFreeFlyer())

## Setup the model to have the same dynamic parameters

In [2]:
with np.printoptions(precision=5, suppress=True, linewidth=400):
    for body_id in mjmodel.jnt_bodyid:
        mjparams = parameters.get_dynamic_parameters(mjmodel, body_id)
        pinparams = pinmodel.inertias[int(body_id)].toDynamicParameters()

        print(mjparams)
        print(pinparams)
        if not np.allclose(mjparams, pinparams, atol=1e-6):
            parameters.set_dynamic_parameters(mjmodel, body_id, pinparams)

            mjparams = parameters.get_dynamic_parameters(mjmodel, body_id)
            print("changed to", mjparams)

[ 5.39    -0.00108  0.00022 -0.24374  0.0556   0.00009  0.01927 -0.00007  0.00001  0.04902]
[ 5.39    -0.00108  0.00022 -0.24374  0.0556   0.00009  0.01927 -0.00007  0.00001  0.04902]
[ 2.244   -0.11047  0.00022  0.01616  0.00269  0.00002  0.0086   0.00028  0.       0.00773]
[ 2.244   -0.11047  0.00022  0.01616  0.00269  0.00002  0.0086   0.00028  0.       0.00773]
[ 2.232   -0.01295 -0.00712 -0.0002   0.00208 -0.00001  0.00232  0.      -0.00001  0.00253]
[ 2.232   -0.01295 -0.00712 -0.0002   0.00208 -0.00001  0.00232  0.      -0.00001  0.00253]
[ 4.152    0.03097 -0.09741 -0.34017  0.11277  0.00006  0.10968  0.00661 -0.00078  0.00852]
[ 4.152    0.03097 -0.09741 -0.34017  0.11277  0.00006  0.10968  0.00661 -0.00078  0.00852]
[ 1.721   -0.00234 -0.00881 -0.23819  0.04522 -0.00008  0.04548  0.00076 -0.00099  0.00211]
[ 1.721   -0.00234 -0.00881 -0.23819  0.04522 -0.00008  0.04548  0.00076 -0.00099  0.00211]
[ 0.446    0.02998  0.00007 -0.02006  0.00116 -0.       0.00513  0.00163  0.    

In [3]:
q = np.random.randn(mjmodel.nq)
v = np.random.randn(mjmodel.nv)
dv = np.random.randn(mjmodel.nv)
tau = np.random.randn(mjmodel.nu)

mjdata.qpos[:] = q
mjdata.qvel[:] = v
mjdata.qacc[:] = dv
mjdata.ctrl[:] = tau

mujoco.mj_step(mjmodel, mjdata)
mujoco.mj_inverse(mjmodel, mjdata)

In [4]:
# setup same data in pinocchio
pinq, pinv = muj2pin(mjdata.qpos, mjdata.qvel)

In [5]:
from mujoco_sysid.regressors import joint_body_regressor

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(joint_body_regressor(mjmodel, mjdata, 2))

[[ -26.04972   -2.26343  951.46826  -99.99949    0.         0.         0.         0.         0.         0.     ]
 [   5.94784 -950.59184   -3.00354   68.78174    0.         0.         0.         0.         0.         0.     ]
 [  28.77278  102.78788  -67.48682   -1.14711    0.         0.         0.         0.         0.         0.     ]
 [   0.         0.        28.77278   -5.94784  -68.13428 -102.78788   -0.64746 -950.59184   -1.85643    0.64746]
 [   0.       -28.77278    0.       -26.04972    1.39419  -67.48682 -101.39368    1.11632 -951.46826   -1.39419]
 [   0.         5.94784   26.04972    0.        -0.43821    0.74011    0.43821  -68.78174  -99.99949 -951.03005]]


In [6]:
pindata = pin.Data(pinmodel)
tau = pin.rnea(pinmodel, pindata, pinq, pinv, mjdata.qacc.copy())

with np.printoptions(precision=5, suppress=True, linewidth=400):
    print(pin.jointBodyRegressor(pinmodel, pindata, 2))

[[ -27.22494   -2.26343  951.46826  -99.99949    0.         0.         0.         0.         0.         0.     ]
 [  19.5201  -950.59184   -3.00354   68.78174    0.         0.         0.         0.         0.         0.     ]
 [  28.80256  102.78788  -67.48682   -1.14711    0.         0.         0.         0.         0.         0.     ]
 [   0.         0.        28.80256  -19.5201   -68.13428 -102.78788   -0.64746 -950.59184   -1.85643    0.64746]
 [   0.       -28.80256    0.       -27.22494    1.39419  -67.48682 -101.39368    1.11632 -951.46826   -1.39419]
 [   0.        19.5201    27.22494    0.        -0.43821    0.74011    0.43821  -68.78174  -99.99949 -951.03005]]
